In [1]:
%cd semantic_code_search/code_serach/modeling

/teamspace/studios/this_studio/semantic_code_search/code_serach/modeling


In [2]:
from omegaconf import OmegaConf
from enum import Enum, auto
from abc import ABC, abstractmethod
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from peft import (PeftModel, PeftConfig, PrefixTuningConfig, 
                 LoraConfig, get_peft_model, TaskType)
from omegaconf import DictConfig
from typing import Optional, Dict, Type
from model_manager import *

In [57]:
config = OmegaConf.load("/teamspace/studios/this_studio/semantic_code_search/conf/modeling.yaml")

In [70]:
model = Model(config.model.model_id)

In [59]:
model_manager = ModelManager()

In [60]:
model = model_manager.prepare_model(model=model,fine_tuning_type=FineTuningType.FULL,config=config.model.fine_tuning,peft_type=PEFTType.LORA)

In [77]:

prepared_model = model_manager.prepare_model(model=model,
                                                config=config.model.fine_tuning,
                                                fine_tuning_type=FineTuningType.FULL,
                                                peft_type=PEFTType.PREFIX)
assert isinstance(prepared_model, torch.nn.Module)
assert hasattr(prepared_model, 'base_model')
for name, param in prepared_model.named_parameters():
    print(name, param.requires_grad)
    # if "prompt_encoder" not in name:
    #     assert param.requires_grad == False

embeddings.word_embeddings.weight False
embeddings.token_type_embeddings.weight False
embeddings.LayerNorm.weight False
embeddings.LayerNorm.bias False
encoder.layer.0.attention.self.query.weight False
encoder.layer.0.attention.self.query.bias False
encoder.layer.0.attention.self.key.weight False
encoder.layer.0.attention.self.key.bias False
encoder.layer.0.attention.self.value.weight False
encoder.layer.0.attention.self.value.bias False
encoder.layer.0.attention.self.layer_norm_q.weight False
encoder.layer.0.attention.self.layer_norm_q.bias False
encoder.layer.0.attention.self.layer_norm_k.weight False
encoder.layer.0.attention.self.layer_norm_k.bias False
encoder.layer.0.attention.output.dense.weight False
encoder.layer.0.attention.output.dense.bias False
encoder.layer.0.attention.output.LayerNorm.weight False
encoder.layer.0.attention.output.LayerNorm.bias False
encoder.layer.0.layer_norm_1.weight False
encoder.layer.0.layer_norm_1.bias False
encoder.layer.0.layer_norm_2.weight Fals

In [79]:
model.encoder

JinaBertModel(
  (embeddings): JinaBertEmbeddings(
    (word_embeddings): Embedding(61056, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): JinaBertEncoder(
    (layer): ModuleList(
      (0-11): 12 x JinaBertLayer(
        (attention): JinaBertAttention(
          (self): JinaBertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (layer_norm_q): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (layer_norm_k): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): JinaBertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, 

In [72]:
num_layers = len([name for name in prepared_model.named_modules() if "encoder.layer" in name])

In [76]:
print([name for name in prepared_model.named_modules() if "encoder.layer" in name])

[('encoder.layer', ModuleList(
  (0-11): 12 x JinaBertLayer(
    (attention): JinaBertAttention(
      (self): JinaBertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (layer_norm_q): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_norm_k): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (output): JinaBertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (layer_norm_1): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (layer_norm_2): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (mlp): JinaBertGLUMLP(
      (act): GELU(approxim

In [46]:

for name, param in model.named_parameters():
    if any(layer in name for layer in trainable_layers):
        assert param.requires_grad == True
    elif "embeddings" in name:
        assert param.requires_grad == False
    elif param.requires_grad:
        assert False

AssertionError: 

In [37]:

for name, param in model.named_parameters():
    if "embeddings" not in name:
        assert param.requires_grad == True
    else:
        assert param.requires_grad == False